In [1]:
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
import nibabel as nib
from os.path import join as opj
from numpy import median, rint

from nipype.interfaces import freesurfer as fs
from nipype.interfaces import spm
from nipype.interfaces import fsl
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.pipeline.engine import Workflow, Node, MapNode

# MATLAB - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

#other study-specific variables
project_home = '/Users/catcamacho/Dropbox/Projects/TH_NAR_ASL/proc/dev'
subjects_dir = project_home
subjects_list = ['ko']
raw_dir = 'raw'
output_dir = 'proc'

fs_dir = subjects_dir + '/freesurfer'
template = project_home + '/template/avg152T1.nii'
#fs.FSCommand.set_default_subjects_dir('/Volumes/iang/users/fischerAS/NART2_rest/NART2_freesurfer/subjDir')
fs.FSCommand.set_default_subjects_dir(fs_dir)

In [4]:
#anat nodes

# Select subjects
infosource = Node(IdentityInterface(fields=['subjid']),
                  name="infosource")
infosource.iterables = [('subjid', subjects_list)]

# SelectFiles
templates = {'cbf': subjects_dir + '/scannerproc/{subjid}/cbf.nii.gz'}
selectfiles = Node(SelectFiles(templates), name="selectfiles")

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

# BBRegister - coregister a volume to the Freesurfer anatomical
bbregister = Node(fs.BBRegister(init='header',
                             contrast_type='t2',
                             out_fsl_file=True),
                  name='bbregister')

# Volume Transformation - transform the brainmask into functional space
applyVolTrans = Node(fs.ApplyVolTransform(inverse=True),
                     name='applyVolTrans')

# Binarize -  binarize and dilate an image to create a brainmask
binarize = Node(fs.Binarize(min=0.5,
                         dilate=1,
                         out_type='nii'),
                name='binarize')

# Realign
realign = Node(spm.Realign(register_to_mean=True),
               name="realign")

# Datasink
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

In [6]:
# Create a preprocessing workflow
anatprocflow = Workflow(name='anatprocflow')

# Connect all components of the preprocessing workflow
anatprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                      (selectfiles, bbregister, [('cbf', 'source_file')]),
                      (fssource, bbregister, [('brainmask', 'target_file')]),
                      (bbregister, applyVolTrans, [('out_reg_file', 'reg_file')]),
                      (realign, applyVolTrans, [('mean_image', 'source_file')]),
                      (applyVolTrans, binarize, [('transformed_file', 'in_file')]),
                     ])
anatprocflow.write_graph(graph2use='flat')
anatprocflow.run('MultiProc', plugin_args={'n_procs': 2})

Exception: Some connections were not found
Module bbregister has no input called target_file


In [5]:
# cbfprcoflow nodes

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subjid']),
                  name="infosource")
infosource.iterables = [('subjid', subjects_list)]

# SelectFiles
templates = {'func': subjects_dir + '/raw/{subjid}/cbf.nii.gz'}
selectfiles = Node(SelectFiles(templates), name="selectfiles")

#gunzip
gunzip = Node(Gunzip(), name="gunzip")

#spatial normalization
norm = Node(spm.Normalize(template=template), name="norm")

# Datasink
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

In [6]:

cbfprocflow = Workflow(name="cbfprocflow")
## Definitely needs to register to anat then register anat to template then apply same transform to cbf. 
## There is not cutting out those steps sadly.

cbfprocflow.base_dir = opj(subjects_dir)
cbfprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                     (selectfiles, gunzip, [('func', 'in_file')]),
                     (gunzip, norm,[('out_file','source')]),
                     (norm, datasink, [('normalized_source', 'norm')]),
                    ])
cbfprocflow.write_graph(graph2use='flat')
cbfprocflow.run('MultiProc', plugin_args={'n_procs': 2})



170228-11:55:35,478 workflow INFO:
	 Creating detailed dot file: /Users/myelin/Documents/asl_practice/subs/cbfprocflow/graph_detailed.dot
170228-11:55:35,569 workflow INFO:
	 Creating dot file: /Users/myelin/Documents/asl_practice/subs/cbfprocflow/graph.dot
170228-11:55:35,642 workflow INFO:
	 ['check', 'execution', 'logging']
170228-11:55:35,654 workflow INFO:
	 Running in parallel.
170228-11:55:35,657 workflow INFO:
	 Executing: selectfiles.a0 ID: 0
170228-11:55:35,669 workflow INFO:
	 Executing node selectfiles.a0 in dir: /Users/myelin/Documents/asl_practice/subs/cbfprocflow/_subjid_ko/selectfiles
170228-11:55:35,694 workflow INFO:
	 Runtime memory and threads stats unavailable
170228-11:55:35,698 workflow INFO:
	 [Job finished] jobname: selectfiles.a0 jobid: 0
170228-11:55:35,702 workflow INFO:
	 Executing: gunzip.a0 ID: 1
170228-11:55:35,713 workflow INFO:
	 Executing node gunzip.a0 in dir: /Users/myelin/Documents/asl_practice/subs/cbfprocflow/_subjid_ko/gunzip
170228-11:55:35,765